In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils

In [2]:
from keras.models import Model, load_model
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, Flatten
from keras.models import Model

Using TensorFlow backend.


In [3]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [4]:
word2ind, ind2word = utils.load_index_word_map()

In [5]:
num_encoder_tokens = len(word2ind)
num_decoder_tokens = len(word2ind)
latent_dim = 100

In [6]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [8]:
encoder_input_data_2d = pd.read_csv('encoder_input.csv').values
decoder_input_data_2d = pd.read_csv('decoder_input.csv').values
decoder_target_data_2d = pd.read_csv('decoder_target.csv').values
# decoder_target_data = decoder_target_data.reshape((decoder_target_data.shape[0], decoder_target_data.shape[1], -1))

In [9]:
def train_valid_split(length, test_size=0.3, random_state=43):
    np.random.RandomState(seed=random_state)
    choices = list(range(length))
    val_choices = np.random.choice(
        choices,
        int(length*test_size),
        replace=False
    ).tolist()
    train_choices = list(set(choices) - set(val_choices))
    return train_choices, val_choices

In [10]:
encoder_input_data_2d.shape

(397639, 40)

In [11]:
def make_3d_one_hot(input_2d, n_tokens):
    z_coords = input_2d.flatten()
    y_mesh, x_mesh = np.meshgrid(list(range(input_2d.shape[1])), list(range(input_2d.shape[0])))
    x_coords = x_mesh.flatten()
    y_coords = y_mesh.flatten()
    input_data = np.zeros((*input_2d.shape, n_tokens))
    input_data[x_coords, y_coords, z_coords] = 1
    return input_data

In [12]:
encoder_input_data = make_3d_one_hot(encoder_input_data_2d, num_encoder_tokens)
decoder_input_data = make_3d_one_hot(decoder_input_data_2d, num_decoder_tokens)
decoder_target_data = make_3d_one_hot(decoder_target_data_2d, num_decoder_tokens)

In [13]:
encoder_input_data.shape

(397639, 40, 76)

In [14]:
decoder_input_data.shape

(397639, 41, 76)

In [15]:
decoder_target_data.shape

(397639, 41, 76)

In [16]:
train_set, val_set = train_valid_split(encoder_input_data.shape[0])

In [17]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, 76)      0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, None, 76)      0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    [(None, 100), (None,  70800       input_1[0][0]                    
____________________________________________________________________________________________________
lstm_2 (LSTM)                    [(None, None, 100), ( 70800       input_2[0][0]                    
                                                                   lstm_1[0][1]            

In [30]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=10000,
          epochs=1,
          validation_split=0.2)

Train on 318111 samples, validate on 79528 samples
Epoch 1/1


KeyboardInterrupt: 

In [31]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [21]:
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')

In [34]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, word2ind[' ']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = ind2word[str(sampled_token_index)]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > 40):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [35]:
decode_sequence(encoder_input_data[:1])

'                                        \n'